In [1]:
type state = {
  grid : int array array;
  mutable i : int;
  mutable j : int;
  mutable h : int;
}

type state = {
  grid : int array array;
  mutable i : int;
  mutable j : int;
  mutable h : int;
}

In [2]:
let s = { (* exemple de l'énoncé *)
    grid =
        [| [| 2; 3; 1; 6 |];
           [| 14; 5; 8; 4 |];
           [| 15; 12; 7; 9 |];
           [| 10; 13; 11; 0|] |];
    i = 2;
    j = 0;
    h = 0
}
let final = (* l'état dans lequel on doit aboutir *)
  let m = Array.make_matrix 4 4 0 in
  for i = 0 to 3 do
    for j = 0 to 3 do
      m.(i).(j) <- i * 4 + j
    done
  done;
  {grid = m; i = 3; j = 3; h = 0}

val s : state =
  {grid =
    [|[|2; 3; 1; 6|]; [|14; 5; 8; 4|]; [|15; 12; 7; 9|]; [|10; 13; 11; 0|]|];
   i = 2; j = 0; h = 0}
val final : state =
  {grid =
    [|[|0; 1; 2; 3|]; [|4; 5; 6; 7|]; [|8; 9; 10; 11|]; [|12; 13; 14; 15|]|];
   i = 3; j = 3; h = 0}

In [3]:
type direction = U | D | L | R

type direction = U | D | L | R

In [21]:
let possible_moves s =
    let l = ref [] in
    if s.i < 3 then l := [D];
    if s.i > 0 then l := U::!l;
    if s.j < 3 then l := R::!l;
    if s.j > 0 then l := L::!l;
    !l

val possible_moves : state -> direction list = <fun>

In [9]:
let compute_h s =
    let h = ref 0 in
    for i = 0 to 3 do
        for j = 0 to 3 do
            let v = s.grid.(i).(j) in
            h := !h + (abs (i - v/4)) + (abs (j - v mod 4))
        done
    done;
    s.h <- !h;;
    
compute_h s;;
s.h

val compute_h : state -> unit = <fun>
- : unit = ()
- : int = 38

In [11]:
let delta_h s d = 
    let (di, dj) = match d with
        | U -> (-1, 0)
        | D -> (1, 0)
        | L -> (0, -1)
        | R -> (0, 1) in
    let i, j = s.i, s.j in
    let v = s.grid.(i + di).(j + dj) in
    (abs (i - v/4)) + (abs (j - v mod 4)) -
    (abs ((i+di) - v/4)) + (abs ((j+dj) - v mod 4))

val delta_h : state -> direction -> int = <fun>

In [42]:
let apply s d =
    let (di, dj) = match d with
        | U -> (-1, 0)
        | D -> (1, 0)
        | L -> (0, -1)
        | R -> (0, 1) in
    let i, j = s.i, s.j in
    let x = s.grid.(i + di).(j + dj) in
    s.grid.(i + di).(j + dj) <- 15;
    s.h <- s.h + delta_h s d;
    s.grid.(i).(j) <- x;
    s.i <- i + di;
    s.j <- j + dj

val apply : state -> direction -> unit = <fun>

In [17]:
let copy s = {
    i = s.i;
    j = s.j;
    h = s.h;
    grid = Array.map (Array.copy) s.grid
}

val copy : state -> state = <fun>

In [33]:
let successors s =
    let rec aux = function
        | [] -> []
        | d::q -> 
            let s' = copy s in
            apply s' d;
            s'::aux q in
    aux (possible_moves s)

val successors : state -> state list = <fun>

In [25]:
type 'a abr = V | N of 'a * 'a abr * 'a abr

let rec add a p e = match a with
    | V -> N((p, e), V, V)
    | N(r, g, d) -> if p < fst r then N(r, add g p e, d)
        else  N(r, g, add d p e)
        
let rec extract_min = function
    | V -> failwith "vide"
    | N(r, V, d) -> r, d
    | N(r, g, d) -> 
        let x, g' = extract_min g in
        x, N(r, g', d)

type 'a abr = V | N of 'a * 'a abr * 'a abr
val add : ('a * 'b) abr -> 'a -> 'b -> ('a * 'b) abr = <fun>
val extract_min : 'a abr -> 'a * 'a abr = <fun>

In [81]:
let a_star s =
    let q = ref V in
    q := add !q s.h s;
    let d = Hashtbl.create 42 in
    while not (Hashtbl.mem d final) do
        let (p, e), q' = extract_min !q in
        q := q';
        Printf.printf "%d " p;
        if not (Hashtbl.mem d e) then (
            let de = p - e.h in
            Hashtbl.add d e de;
            List.iter (fun s ->
                q := add !q (de + 1 + s.h) s
            ) (successors e)
        )
    done;
    Hashtbl.find d final

val a_star : state -> int = <fun>

In [82]:
a_star ten;;

-1 -1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 11 11 

Stack overflow during evaluation (looping recursion?).

In [47]:
successors ten

- : state list =
[{grid =
   [|[|0; 1; 2; 3|]; [|4; 5; 6; 7|]; [|8; 9; 10; 10|]; [|12; 13; 14; 11|]|];
  i = 2; j = 2; h = 0};
 {grid =
   [|[|0; 1; 2; 3|]; [|4; 5; 6; 7|]; [|8; 9; 10; 7|]; [|12; 13; 14; 11|]|];
  i = 1; j = 3; h = 0};
 {grid =
   [|[|0; 1; 2; 3|]; [|4; 5; 6; 7|]; [|8; 9; 10; 11|]; [|12; 13; 14; 11|]|];
  i = 3; j = 3; h = -2}]

In [79]:
let ten =
  let moves = [U] in
  let state = copy final in
  List.iter (apply state) moves;
  state


val ten : state =
  {grid =
    [|[|0; 1; 2; 3|]; [|4; 5; 6; 7|]; [|8; 9; 10; 15|]; [|12; 13; 14; 11|]|];
   i = 2; j = 3; h = -1}

In [51]:
ten

- : state =
{grid =
  [|[|0; 1; 2; 3|]; [|4; 5; 6; 7|]; [|8; 9; 10; 11|]; [|12; 13; 14; 15|]|];
 i = 3; j = 3; h = 0}

In [50]:
apply ten D;

- : unit = ()